In [1]:
#| default_exp adv_perturb

In [1]:
#| hide
!nvidia-smi

Fri Oct 27 17:19:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:B1:00.0 Off |                  N/A |
| 32%   54C    P2              79W / 250W |    807MiB / 11264MiB |     36%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
#| hide
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2,3"

In [4]:
#| hide
ngpu = torch.cuda.device_count()
for gpu_id in range(ngpu):
    gpu = torch.cuda.get_device_name(gpu_id)
    print(f"GPU:{gpu_id} {gpu}")

GPU:0 NVIDIA GeForce RTX 2080 Ti
GPU:1 NVIDIA GeForce RTX 2080 Ti


In [10]:
#| export
from typing import Any, Callable, Optional, Tuple, Union

import torch
from captum._utils.common import (
    _format_additional_forward_args,
    _format_output,
    _format_tensor_into_tuples,
    _is_tuple,
    _select_targets,
)
from captum._utils.gradient import (
    apply_gradient_requirements,
    compute_gradients,
    undo_gradient_requirements,
)
from captum._utils.typing import TensorOrTupleOfTensorsGeneric
from captum.log import log_usage
from captum.robust._core.perturbation import Perturbation
from torch import Tensor

class FGSM_MOD(Perturbation):
    def __init__(
        self,
        forward_func: Callable,
        loss_func: Optional[Callable] = None,
        lower_bound: float = float("-inf"),
        upper_bound: float = float("inf"),
        anti_adv: bool = False
    ) -> None:
        super().__init__()
        self.forward_func = forward_func
        self.loss_func = loss_func
        self.bound = lambda x: torch.clamp(x, min=lower_bound, max=upper_bound)
        self.zero_thresh = 10**-6
        self.anti_adv = anti_adv

    @log_usage()
    def perturb(
        self,
        inputs: TensorOrTupleOfTensorsGeneric,
        epsilon: float,
        target: Any,
        additional_forward_args: Any = None,
        targeted: bool = False,
        mask: Optional[TensorOrTupleOfTensorsGeneric] = None,
    ) -> TensorOrTupleOfTensorsGeneric:
        is_inputs_tuple = _is_tuple(inputs)
        inputs: Tuple[Tensor, ...] = _format_tensor_into_tuples(inputs)
        masks: Union[Tuple[int, ...], Tuple[Tensor, ...]] = (
            _format_tensor_into_tuples(mask)
            if (mask is not None)
            else (1,) * len(inputs)
        )
        gradient_mask = apply_gradient_requirements(inputs)

        def _forward_with_loss() -> Tensor:
            additional_inputs = _format_additional_forward_args(additional_forward_args)
            outputs = self.forward_func(  # type: ignore
                *(*inputs, *additional_inputs)  # type: ignore
                if additional_inputs is not None
                else inputs
            )
            if self.loss_func is not None:
                return self.loss_func(outputs, target)
            else:
                loss = -torch.log(outputs)
                return _select_targets(loss, target)

        grads = compute_gradients(_forward_with_loss, inputs)
        undo_gradient_requirements(inputs, gradient_mask)
        if self.anti_adv:
            perturbed_inputs = self._anti_perturb(inputs, grads, epsilon, targeted, masks)
        else:
            perturbed_inputs = self._perturb(inputs, grads, epsilon, targeted, masks)
        perturbed_inputs = tuple(
            self.bound(perturbed_inputs[i]) for i in range(len(perturbed_inputs))
        )
        return _format_output(is_inputs_tuple, perturbed_inputs)

    def _perturb(
        self,
        inputs: Tuple,
        grads: Tuple,
        epsilon: float,
        targeted: bool,
        masks: Tuple,
    ) -> Tuple:
        multiplier = -1 if targeted else 1
        inputs = tuple(
            torch.where(
                torch.abs(grad) > self.zero_thresh,
                inp + multiplier * epsilon * torch.sign(grad) * mask,
                inp,
            )
            for grad, inp, mask in zip(grads, inputs, masks)
        )
        return inputs
    
    def _anti_perturb(
        self,
        inputs: Tuple,
        grads: Tuple,
        epsilon: float,
        targeted: bool,
        masks: Tuple,
    ) -> Tuple:
        multiplier = -1 if targeted else 1
        inputs = tuple(
            torch.where(
                torch.abs(grad) > self.zero_thresh,
                inp - multiplier * epsilon * torch.sign(grad) * mask,
                inp,
            )
            for grad, inp, mask in zip(grads, inputs, masks)
        )
        return inputs

### Adversarial Attack

In [11]:
#| export
def adversarial_noise(inputs, model, class_idx, epsilon, lower_bound=-1, upper_bound=1, anti_adv=False, mask=None):
    """
    Adds (anti-)adversarial noise to an image and returns the modified image.
    
    input :
      - inputs : torch.tensor (A torch tensor with shape (channels, height, width))
      - model : function (Model function which output probability)
      - class_idx : int (Index of the class based on which we apply (anti-)adversarial noise)
      - epsilon : float32 (Intensity of the noise added, lower value changes the image less)
      - lower_bound : float32 (The minimum value below which we would clamp)
      - upper_bound : float32 (The maximum value above which we would clamp)
      - anti_adv : boolean (Should the method attack the class_idx)
      - mask : ndarray (Array of float32 or int32, modulating the noises added based on their values)
      
    return :
      - perturbed_img : torch.tensor (Tensor containing the perturbed image)
    """
    fgsm = FGSM_MOD(model, lower_bound=lower_bound, upper_bound=upper_bound, anti_adv=anti_adv)
    perturbed_img = fgsm.perturb(inputs, epsilon=epsilon, target=class_idx, mask=mask)
    return perturbed_img

In [1]:
#| hide
from nbdev.export import nb_export
nb_export('adversarial_perturbation.ipynb', '../../commons/api/method')